In [ ]:
### Create ChArUco marker

import os
import numpy as np
import cv2

# ------------------------------
# ENTER YOUR PARAMETERS HERE:
ARUCO_DICT = cv2.aruco.DICT_6X6_250
SQUARES_VERTICALLY = 7
SQUARES_HORIZONTALLY = 5
SQUARE_LENGTH = 0.03
MARKER_LENGTH = 0.015
LENGTH_PX = 640   # total length of the page in pixels
MARGIN_PX = 20    # size of the margin in pixels
SAVE_NAME = 'ChArUco_Marker.png'
# ------------------------------

def create_and_save_new_board():
    dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
    board = cv2.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
    size_ratio = SQUARES_HORIZONTALLY / SQUARES_VERTICALLY
    img = cv2.aruco.CharucoBoard.generateImage(board, (LENGTH_PX, int(LENGTH_PX*size_ratio)), marginSize=MARGIN_PX)
    cv2.imshow("img", img)
    cv2.imwrite(SAVE_NAME, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

create_and_save_new_board()

In [18]:
### calibrate camera using ChArUro
import cv2  
import json
import os

ARUCO_DICT = cv2.aruco.DICT_6X6_250
SQUARES_VERTICALLY = 7
SQUARES_HORIZONTALLY = 5
SQUARE_LENGTH = 0.03
MARKER_LENGTH = 0.015
   

def get_calibration_parameters(img_dir):
    # Define the aruco dictionary, charuco board and detector
    dictionary = cv2.aruco.getPredefinedDictionary(ARUCO_DICT)
    board = cv2.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
    params = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, params)

    # Load images from directory
    image_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(".jpg")]
    all_charuco_ids = []
    all_charuco_corners = []

    # Loop over images and extraction of corners
    for image_file in image_files:
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        imgSize = image.shape
        image_copy = image.copy()
        marker_corners, marker_ids, rejectedCandidates = detector.detectMarkers(image)
        
        if len(marker_ids) > 0: # If at least one marker is detected
            # cv2.aruco.drawDetectedMarkers(image_copy, marker_corners, marker_ids)
            ret, charucoCorners, charucoIds = cv2.aruco.interpolateCornersCharuco(marker_corners, marker_ids, image, board)

            if charucoIds is not None and len(charucoCorners) > 3:
                all_charuco_corners.append(charucoCorners)
                all_charuco_ids.append(charucoIds)
    
    # Calibrate camera with extracted information
    result, mtx, dist, rvecs, tvecs = cv2.aruco.calibrateCameraCharuco(all_charuco_corners, all_charuco_ids, board, imgSize, None, None)
    return mtx, dist,rvecs,tvecs


root = os.getcwd()
OUTPUT_JSON = os.path.join(root, 'charuco_calib', 'calib_data','calib.json')
imgs_dir = os.path.join(root, 'charuco_calib','calib_images')
mtx, dist,rvecs,tvecs = get_calibration_parameters(img_dir=imgs_dir)
data = {"mtx": mtx.tolist(), "dist": dist.tolist()}

with open(OUTPUT_JSON, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f'Data has been saved to {OUTPUT_JSON}')


print("dumping the data into one files using numpy ")
np.savez(
    os.path.join(root, 'charuco_calib', 'calib_data','MultiMatrix'),
    camMatrix=mtx,
    distCoef=dist,
    rVector=rvecs,
    tVector=tvecs,
)

Data has been saved to e:\Abhi\Documents\IITMadras\Torso Stabilisation\pose_depth\Realtime_object_depth_estimator\charuco_calib\calib_data\calib.json
dumping the data into one files using numpy 


In [ ]:
#Capture images 
import os
import glob
import cv2
import numpy as np
# Define the save directory
SAVE_DIR = glob.glob('charuco_calib')[0]
os.makedirs(SAVE_DIR, exist_ok=True)  # Create directory if not exists

# Open webcam
cap = cv2.VideoCapture(3)  # 0 = Default webcam

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Press SPACE to capture an image. Press ESC to exit.")

image_count = 1  # Counter for image filenames

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Show the live webcam feed
    cv2.imshow("Webcam - Press SPACE to Capture", frame)

    key = cv2.waitKey(1) & 0xFF

    # Press SPACE to capture an image
    if key == 32:  # Spacebar key
        image_path = os.path.join(SAVE_DIR, f"captured_{image_count}.jpg")
        cv2.imwrite(image_path, frame)
        print(f"Image saved: {image_path}")
        image_count += 1

    # Press ESC to exit
    elif key == 27:  # ESC key
        print("Exiting...")
        break

# Release webcam and close windows
cap.release()
cv2.destroyAllWindows()


Press SPACE to capture an image. Press ESC to exit.
Image saved: charuco_calib\captured_1.jpg
Image saved: charuco_calib\captured_2.jpg
Image saved: charuco_calib\captured_3.jpg
Image saved: charuco_calib\captured_4.jpg
Image saved: charuco_calib\captured_5.jpg
Image saved: charuco_calib\captured_6.jpg
Image saved: charuco_calib\captured_7.jpg
Image saved: charuco_calib\captured_8.jpg
Image saved: charuco_calib\captured_9.jpg
Image saved: charuco_calib\captured_10.jpg
Image saved: charuco_calib\captured_11.jpg
Image saved: charuco_calib\captured_12.jpg
Image saved: charuco_calib\captured_13.jpg
Image saved: charuco_calib\captured_14.jpg
Image saved: charuco_calib\captured_15.jpg
Image saved: charuco_calib\captured_16.jpg
Image saved: charuco_calib\captured_17.jpg
Image saved: charuco_calib\captured_18.jpg
Image saved: charuco_calib\captured_19.jpg
Image saved: charuco_calib\captured_20.jpg
Image saved: charuco_calib\captured_21.jpg
Image saved: charuco_calib\captured_22.jpg
Image saved

In [ ]:
### Detect ArUco markers
import cv2
from cv2 import aruco
import numpy as np

marker_dict  = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)

param_markers = aruco.DetectorParameters()

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    marker_corners, marker_IDs, reject = aruco.detectMarkers(
        gray_frame, marker_dict, parameters = param_markers
    )
    if marker_corners:
        for i in range(len(marker_corners)):
            cv2.polylines(
                frame, [marker_corners[i].astype(np.int32)], True, (0,255,0), 4
            )
            corners = marker_corners[i].reshape(4,2)
            corners = corners.astype(int)
            top_right = corners[0].ravel()
            top_left = corners[1].ravel()
            bottom_right = corners[2].ravel()
            bottom_left = corners[3].ravel()
            cv2.putText(
                frame, 
                f"id: {marker_IDs[i][0]}",
                top_right,
                cv2.FONT_HERSHEY_PLAIN,
                1.3,
                (200,100,0),
                2,
                cv2.LINE_AA
            )
    cv2.imshow("frame", frame)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

In [21]:
### Distance estimation using ArUco markers

import cv2 as cv
from cv2 import aruco
import numpy as np

# load in the calibration data
root = os.getcwd()
calib_data_path = os.path.join(root,'charuco_calib','calib_data','MultiMatrix.npz') 

calib_data = np.load(calib_data_path)
print(calib_data.files)

cam_mat = calib_data["camMatrix"]
dist_coef = calib_data["distCoef"]
r_vectors = calib_data["rVector"]
t_vectors = calib_data["tVector"]

MARKER_SIZE = 4.6  # centimeters (measure your printed marker size)

marker_dict = aruco.getPredefinedDictionary(aruco.DICT_6X6_250)

param_markers = aruco.DetectorParameters()

cap = cv.VideoCapture(3)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    marker_corners, marker_IDs, reject = aruco.detectMarkers(
        gray_frame, marker_dict, parameters=param_markers
    )
    if marker_corners:
        rVec, tVec, _ = aruco.estimatePoseSingleMarkers(
            marker_corners, MARKER_SIZE, cam_mat, dist_coef
        )
        total_markers = range(0, marker_IDs.size)
        for ids, corners, i in zip(marker_IDs, marker_corners, total_markers):
            cv.polylines(
                frame, [corners.astype(np.int32)], True, (0, 255, 255), 4, cv.LINE_AA
            )
            corners = corners.reshape(4, 2)
            corners = corners.astype(int)
            top_right = corners[0].ravel()
            top_left = corners[1].ravel()
            bottom_right = corners[2].ravel()
            bottom_left = corners[3].ravel()

            # Calculating the distance
            distance = np.sqrt(
                tVec[i][0][2] ** 2 + tVec[i][0][0] ** 2 + tVec[i][0][1] ** 2
            )
            # Draw the pose of the marker
            point = cv.drawFrameAxes(frame, cam_mat, dist_coef, rVec[i], tVec[i], 4, 4)
            cv.putText(
                frame,
                f"id: {ids[0]} Dist: {round(distance, 2)}",
                top_right,
                cv.FONT_HERSHEY_PLAIN,
                1.3,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )
            cv.putText(
                frame,
                f"x:{round(tVec[i][0][0],1)} y: {round(tVec[i][0][1],1)} ",
                bottom_right,
                cv.FONT_HERSHEY_PLAIN,
                1.0,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )
            # print(ids, "  ", corners)
    cv.imshow("frame", frame)
    key = cv.waitKey(1)
    if key == ord("q"):
        break
cap.release()
cv.destroyAllWindows()

['camMatrix', 'distCoef', 'rVector', 'tVector']
